# Import Statements

In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.ops import cascaded_union

import matplotlib.pyplot as plt
import geoplot
import seaborn as sns
import plotly.graph_objects as go

# Load Data

In [3]:
os.getcwd()

'C:\\Users\\arjun\\Desktop\\davidson_lab\\india_RE_transition\\notebooks'

In [4]:
os.chdir('../data/tables')

In [5]:
district_emp = pd.read_csv('district_emp_data_full.csv')
district_emp = district_emp.replace(
    {'Jharkhand ': 'Jharkhand',
     'Maharashtra ': 'Maharashtra',
     'Orissa': 'Odisha',
     'Telangana ': 'Telangana'
    }
)
district_emp.head()

,State/UT Name,District Name,Total Coal mining jobs (Direct + Indirect Jobs),Total Power plant jobs (Direct + Indirect Jobs),Total Coal Jobs (Direct + Indirect),Total Pensioners,DMF USD (in Million $),Mining CSR (in lakhs),CSR Power Plant CSR (in lakhs),CSR total (in lakhs),CSR ( in cr),CSR USD (in Million $)
0,Jharkhand,Dhanbad,489394.0,4508.0,493901,70301,37,6688.14,0.00,6688.14,66.8814,9.363396
1,Chhattisgarh,Korba,363591.0,23397.0,386988,14861,62,111.81,954.75,1066.56,10.6656,1.493184
2,West Bengal,Paschim Bardhaman,273408.0,4848.0,278256,65447,1,554.61,0.00,554.61,5.5461,0.776454
3,Jharkhand,Chatra,226763.0,0.0,226763,1996,15,671.49,0.00,671.49,6.7149,0.940086
4,Odisha,Angul,175229.0,18909.0,194139,6613,47,11975.64,1180.00,13155.64,131.5564,18.417896


In [6]:
# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/TDEK8O&version=1.0
pai_coal_mines = pd.read_csv('pai_coal_mines.csv')
pai_coal_mines = pai_coal_mines.replace(
    {'Jharkhand ': 'Jharkhand',
     'Maharashtra ': 'Maharashtra',
     'Orissa': 'Odisha',
     'Telangana ': 'Telangana'
    }
)
pai_coal_mines.head()

,SL No.,State/UT Name,District Name,Mine Name,Coal/ Lignite Production (MT) (2019-2020),Coal Mine Owner Name,Coal/Lignite,Govt Owned/Private,Type of Mine (OC/UG/Mixed),Latitude,Longitude,Source,Accuracy (exact vs approximate),Unnamed: 13
0,1,West Bengal,Paschim Bardhaman,Ningah Colliery,0.01,ECL,Coal,G,UG,23.6743,87.0333,Google Maps: https://www.google.com/maps/place...,Exact,NaN
1,2,West Bengal,Paschim Bardhaman,Jhanjhara Project Colly,3.50,ECL,Coal,G,UG,23.6680,87.2963,Google Maps: https://www.google.ca/maps/place/...,Exact,NaN
2,3,West Bengal,Paschim Bardhaman,MAOHUSUDANPUR 7 PIT &\nINCLINE,0.04,ECL,Coal,G,UG,23.6338,87.2037,Google Maps: https://www.google.com/maps/place...,Approximate: Madhusudanpur Colliery Agent Office,NaN
3,4,West Bengal,Paschim Bardhaman,PARASCOLE(EAST),0.10,ECL,Coal,G,UG,23.6377,87.1951,Google Maps: https://www.google.com/maps/place...,Approximate: center coordinates of eastern Par...,NaN
4,5,West Bengal,Paschim Bardhaman,PARASCOLE(WEST),0.07,ECL,Coal,G,UG,23.6405,87.1855,Google Maps: https://www.google.com/maps/place...,Approximate: center coordinates of western Par...,NaN


In [7]:
plants = pd.read_csv("Global-Coal-Plant-Tracker-Jan-2022.csv", encoding="ISO-8859-1")
plants = plants[plants['Country'] == 'India']
plants = plants[plants['Latitude'].isna() == False]
plants = plants[plants['Longitude'].isna() == False]
plants = plants[plants['Status'].isin(['operating'])] 
plants = plants.drop_duplicates('ParentID')
plants = plants[['Plant', 'Subnational unit (province, state)', 'Status', 'Latitude', 'Longitude', 
                 'Capacity (MW)', 'Annual CO2 (million tonnes / annum)', 'Year']]
plants = plants.rename(columns={'Subnational unit (province, state)': 'State', 'Year': 'Plant Age'})
plants['Capacity (MW)'] = plants['Capacity (MW)'].astype(float)
plants['Plant Age'] = plants['Plant Age'].apply(lambda x: 2021 - float(x))
plants = plants.replace(
    {'Madhya Pradesh / Uttar Pradesh': 'Madhya Pradesh',
     'Punjab state': 'Punjab',
     'Uttar Pradesh / Madhya Pradesh': 'Uttar Pradesh'
    }
)
plants.head()

,Plant,State,Status,Latitude,Longitude,Capacity (MW),Annual CO2 (million tonnes / annum),Plant Age
508,Balaji Cement power station,Andhra Pradesh,operating,16.283333,81.216667,35.0,0.2,12.0
509,Bander power station,Andhra Pradesh,operating,14.351195,80.143086,660.0,2.5,6.0
511,Bhadradri power station,Telangana,operating,18.027488,80.735645,270.0,1.3,1.0
515,Dr Narla Tata Rao power station,Andhra Pradesh,operating,16.598490,80.536585,210.0,1.2,42.0
522,East Godavari Mill power station,Andhra Pradesh,operating,17.300117,82.614386,34.0,0.2,15.0


In [10]:
os.chdir('../pickle')

In [11]:
# district data
file = open("districts.pickle",'rb')
districts = pickle.load(file)
file.close()

In [20]:
districts['District Name']

0          Nainital
1          Dehradun
2            Almora
3         Champawat
4        Uttarkashi
           ...     
687      Wanaparthy
688       Vikarabad
689    Mahabubnagar
690          Kargil
691    Leh (Ladakh)
Name: District Name, Length: 692, dtype: object

In [24]:
692 - 52

640

In [27]:
pai_coal_mines.groupby('District Name').count()['SL No.'].values

array([ 5, 13, 13,  2,  2,  3,  6, 14,  1,  2,  2,  1, 14, 26,  4, 16,  3,
        1, 51,  2,  2, 10,  6,  9,  1,  1,  3, 15, 10,  3,  3, 19,  2, 17,
        2,  2, 65, 14,  2, 10, 19,  4,  3,  7,  7,  3,  5,  3, 10,  3,  7,
       11], dtype=int64)

# Missingness

States missing from Pai Data

In [ ]:
set(plants['State']).union(set(mines['State, Province'])) - set(district_emp['State/UT Name'])

In [ ]:
set(mines['State, Province']) - set(pai_coal_mines['State/UT Name'])

States missing from GEM data

In [ ]:
set(district_emp['State/UT Name']) - set(plants['State']).union(set(mines['State, Province']))

In [ ]:
set(pai_coal_mines['State/UT Name']) - set(mines['State, Province'])

# Visualize Inputs

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../../data/shapes')
transmission = gpd.read_file('transmission.shp')

In [ ]:
os.chdir('../pickle')
file = open("polygons.pickle", "rb")
polygons = pickle.load(file)
file.close()
file = open("states.pickle", "rb")
states = pickle.load(file)
file.close()

In [ ]:
os.chdir('../../outputs/plots')
fig = geoplot.polyplot(
    gpd.GeoSeries(polygons),
    edgecolor='red',
    facecolor='red',
    figsize=(12, 12), 
)

geoplot.polyplot(
    gpd.GeoSeries(states),
    edgecolor='gray',
    ax=fig,
    linewidth=0.75
)

geoplot.sankey(
    gpd.GeoSeries(transmission['geometry']),
    ax=fig
)


fig = fig.get_figure()
fig.savefig('alternative_coal_incumbency_transmission')